# 🧼 02 – Data Cleaning & Validation

This notebook is part of the **Sales Health Monitor** project.

## 🎯 Objective
The goal of this notebook is to:
1. **Assess** the quality of the raw (corrupted) sales dataset.
2. **Detect** issues such as missing values, duplicates, outliers, and incorrect formats.
3. **Clean** the dataset using generalized methods that can be reused in future projects.
4. **Validate** the cleaned dataset to ensure it meets business and data quality standards.
5. **Export** the cleaned dataset for further analysis in Python, MySQL, and Power BI.

## 🔄 Why This Step Matters
Data quality directly impacts business insights and decision-making.  
By applying a systematic cleaning and validation process, we ensure that:
- The analysis phase will run smoothly without errors.
- The ML anomaly detection model will be trained on reliable data.
- This cleaning framework can be reused for future datasets with minimal changes.

---

We will begin with **Step 1 – Setup & Profiling**, which includes:
- Importing required Python libraries.
- Defining project configuration variables.
- Loading the dataset.
- Running initial profiling to understand its structure and potential issues.


## 1.1 Project Setup – Imports & Configuration

In this section:
- Import Python libraries for data loading, profiling, and cleaning.
- Define **configuration variables** such as file paths, column categories, and business rules.

### Why This Matters
By centralising settings here, this notebook becomes:
- **Reusable**: Future datasets only require changes to this section.
- **Easy to maintain**: No need to search through the notebook for hardcoded values.

*(I prefer this approach so I can quickly adapt the cleaning process to new projects without rewriting the code.)*


In [46]:
# ==== 1.1 Import Required Libraries ====
import pandas as pd
import numpy as np
import os

# ==== 1.2 Project Configuration ====
PROJECT_NAME = "Sales Health Monitor"

DATA_PATHS = {
    "raw": "../Dataset/raw/",
    "processed": "../Dataset/processed/",
    "sample": "../Dataset/sample/"
}

DATASET_CONFIGS = {
    "main_dataset": {
        "file": "sales_corrupted.csv",  # File to clean
        "date_columns": ["transaction_date"],  # Columns with date values
        "id_columns": ["transaction_id", "customer_id"],  # Key ID columns
        "numeric_columns": ["quantity", "unit_price", "discount_percent", "total_amount"],
        "categorical_columns": ["customer_segment", "product_category", "region", "sales_channel"]
    }
}

BUSINESS_RULES = {
    "date_range": ("2022-01-01", "2024-12-31"),
    "quantity_range": (0, 1000),
    "unit_price_range": (0, 100000),
    "discount_percent_range": (0, 100),
    "total_amount_range": (0, 1000000)
}


## 1.2 Load Dataset

In this step:
- Load the **raw corrupted dataset** defined in `DATASET_CONFIGS`.
- Display basic information to confirm successful loading.
- Preview the first few rows to understand the structure.

### Why This Matters
Before starting any profiling or cleaning:
- We need to ensure the file path and name are correct.
- A quick preview helps spot obvious problems (e.g., wrong delimiter, unexpected columns).


In [47]:
# ==== Load the dataset based on config ====

# Build full path
dataset_path = os.path.join(DATA_PATHS["raw"], DATASET_CONFIGS["main_dataset"]["file"])

# Read CSV
df = pd.read_csv(dataset_path)

# Show basic info
print(f"✅ {PROJECT_NAME} - Dataset Loaded Successfully")
print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]:,} columns\n")

# Saving a copy
df_raw = df.copy()

# Preview first 5 rows
df.head()


✅ Sales Health Monitor - Dataset Loaded Successfully
📊 Shape: 801,440 rows × 13 columns



,transaction_id,transaction_date,transaction_datetime,customer_id,customer_segment,product_id,product_category,region,sales_channel,quantity,unit_price,discount_percent,total_amount
0,TXN_00000001,2022-01-01,2022-01-01 14:43:00,CUST_022237,Standard,PROD_0302,Sports & Outdoors,South,Mobile App,1,246.29,0.0,246.29
1,TXN_00000002,2022-01-01,2022-01-01 20:50:00,CUST_034632,Budget,PROD_0173,Clothing,East,Retail Store,2,91.23,0.0,182.46
2,TXN_00000003,2022-01-01,2022-01-01 11:18:00,CUST_046496,Standard,PROD_0157,Clothing,West,Online,1,144.35,9.0,144.35
3,TXN_00000004,2022-01-01,2022-01-01 07:52:00,CUST_003169,Budget,PROD_0139,Clothing,East,Mobile App,1,165.72,7.6,165.72
4,TXN_00000005,2022-01-01,2022-01-01 09:33:00,CUST_045584,Standard,PROD_0138,Clothing,West,Mobile App,2,72.16,0.0,144.32


## 1.3 Initial Profiling

In this step:
- Review the dataset’s structure and column data types.
- Identify:
  - Missing values per column.
  - Duplicate rows.
  - Summary statistics for numeric and categorical columns.

### Why This Matters
Initial profiling acts as a **data health check**:
- Confirms that the dataset loaded correctly.
- Highlights the scale and type of quality issues.
- Provides a **baseline** for comparing before vs after cleaning results.

*(I think of this step as the “X-ray” before surgery — it tells us exactly where the problems are.)*


In [48]:
# ==== 1.3 Initial Profiling ====

# --- Dataset Structure & Data Types ---
print("📋 Dataset Info:")
df.info()
print("\n")

# --- Missing Values ---
print("🔍 Missing Values per Column:")
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
print(missing_counts if not missing_counts.empty else "No missing values found.")
print("\n")

# --- Duplicate Rows (exact matches) ---
duplicate_count = df.duplicated().sum()
print(f"🪞 Duplicate Rows (exact matches): {duplicate_count}\n")

# --- Numeric Summary ---
numeric_cols = DATASET_CONFIGS["main_dataset"]["numeric_columns"]
print("📊 Numeric Columns Summary:")
print(df[numeric_cols].describe().T)
print("\n")

# --- Categorical Summary ---
categorical_cols = DATASET_CONFIGS["main_dataset"]["categorical_columns"]
print("📊 Categorical Columns Unique Value Counts:")
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count} unique values")


📋 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801440 entries, 0 to 801439
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   transaction_id        801440 non-null  object 
 1   transaction_date      801440 non-null  object 
 2   transaction_datetime  801440 non-null  object 
 3   customer_id           785399 non-null  object 
 4   customer_segment      801440 non-null  object 
 5   product_id            801440 non-null  object 
 6   product_category      801440 non-null  object 
 7   region                801440 non-null  object 
 8   sales_channel         801440 non-null  object 
 9   quantity              801440 non-null  int64  
 10  unit_price            801440 non-null  float64
 11  discount_percent      801440 non-null  float64
 12  total_amount          801440 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 79.5+ MB


🔍 Missing Values per Column:

## 1.4 Outlier Preview

In this step:
- Identify potential extreme values in numeric columns using the **Interquartile Range (IQR)** method.
- This is only an **early detection** step — we are not removing or adjusting outliers yet.

### Why This Matters
- Confirms whether extreme values seen in summary stats are genuine anomalies.
- Helps guide later cleaning decisions (remove, cap, or keep).


In [49]:
# ==== 1.4 Outlier Preview using IQR Method ====

numeric_cols = DATASET_CONFIGS["main_dataset"]["numeric_columns"]

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
    print(f"{col}: {outlier_count:,} potential outliers")


quantity: 24,860 potential outliers
unit_price: 86,883 potential outliers
discount_percent: 119,878 potential outliers
total_amount: 89,771 potential outliers


# Step 2 – Cleaning Pipeline

In this step:
- Apply data cleaning techniques to address the issues found during Step 1 profiling.
- Focus on:
  1. Missing values.
  2. Out-of-range or invalid numeric values.
  3. Inconsistent categorical values.
  4. Duplicate records.

The goal is to ensure that the dataset:
- Meets the defined business rules.
- Is consistent and free from obvious quality issues.
- Is ready for deeper analysis in later stages.


## 2.1 Handling Missing Values

Missing values can occur for many reasons:
- Data entry errors
- System glitches
- Deliberate masking of unavailable information

### Approach in This Framework
1. **ID columns** (`customer_id`):
   - If missing → drop those rows (IDs are essential for tracking transactions).
2. **Categorical columns**:
   - If missing → replace with `"Unknown"`.
3. **Numeric columns**:
   - If missing → replace with median value (less sensitive to outliers than mean).

This function is flexible:
- It reads column roles from `DATASET_CONFIGS`.
- It allows quick changes in future projects.


In [50]:
def handle_missing_values(df, configs):
    """
    Handles missing values for numeric, categorical, and ID columns
    based on the dataset configuration.
    Prints before/after missing value counts.
    """
    dataset_conf = configs["main_dataset"]
    
    print("🔍 Missing Values BEFORE Cleaning:")
    print(df.isnull().sum()[df.isnull().sum() > 0])
    print("-" * 50)
    
    # Drop rows where ID is missing
    for id_col in dataset_conf["id_columns"]:
        before_rows = df.shape[0]
        df = df[df[id_col].notna()]
        after_rows = df.shape[0]
        dropped = before_rows - after_rows
        if dropped > 0:
            print(f"🗑 Dropped {dropped:,} rows with missing {id_col}")
    
    # Fill missing numeric values with median
    for col in dataset_conf["numeric_columns"]:
        if df[col].isnull().sum() > 0:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"🧮 Filled missing '{col}' with median = {median_val}")
    
    # Fill missing categorical values with 'Unknown'
    for col in dataset_conf["categorical_columns"]:
        if df[col].isnull().sum() > 0:
            df[col].fillna("Unknown", inplace=True)
            print(f"🔤 Filled missing '{col}' with 'Unknown'")
    
    print("-" * 50)
    print("✅ Missing Values AFTER Cleaning:")
    print(df.isnull().sum()[df.isnull().sum() > 0])
    
    return df

# Apply missing value handling
df = handle_missing_values(df, DATASET_CONFIGS)


🔍 Missing Values BEFORE Cleaning:
customer_id    16041
dtype: int64
--------------------------------------------------
🗑 Dropped 16,041 rows with missing customer_id
--------------------------------------------------
✅ Missing Values AFTER Cleaning:
Series([], dtype: int64)


## 2.2 Handling Invalid Values

In this step:
- Enforce **BUSINESS_RULES** for numeric columns.
- Identify and fix:
  1. Negative or zero quantities.
  2. Unrealistic unit prices.
  3. Discounts outside the allowed range.
  4. Total amounts outside realistic bounds.

### Approach:
- For each numeric column, compare values to the min/max from `BUSINESS_RULES`.
- If values are **outside** the range:
  - Quantities → replace with median valid quantity.
  - Prices & totals → replace with median valid value.
  - Discount → cap at valid range (0–25% for this dataset, but can be configured).
- Keep a **log** of how many rows were adjusted.

This ensures:
- Data stays realistic.
- Extreme corrupted valu


In [51]:
def handle_invalid_values(df, configs, rules):
    """
    Fixes numeric values that fall outside the acceptable ranges
    defined in BUSINESS_RULES.
    """
    dataset_conf = configs["main_dataset"]
    
    for col in dataset_conf["numeric_columns"]:
        min_val, max_val = rules[f"{col}_range"] if f"{col}_range" in rules else (None, None)
        
        if min_val is not None and max_val is not None:
            # Find invalid rows
            invalid_mask = (df[col] < min_val) | (df[col] > max_val)
            invalid_count = invalid_mask.sum()
            
            if invalid_count > 0:
                if col == "discount_percent":
                    # Cap discounts at limits
                    df[col] = df[col].clip(lower=min_val, upper=max_val)
                    print(f"🔧 Capped {invalid_count:,} values in '{col}' to range {min_val}–{max_val}")
                else:
                    # Replace invalid numeric values with median of valid ones
                    median_val = df.loc[~invalid_mask, col].median()
                    df.loc[invalid_mask, col] = median_val
                    print(f"🔧 Replaced {invalid_count:,} invalid '{col}' values with median = {median_val}")
            else:
                print(f"✅ No invalid values found in '{col}'")
    
    return df

# Apply invalid value handling
df = handle_invalid_values(df, DATASET_CONFIGS, BUSINESS_RULES)

🔧 Replaced 784 invalid 'quantity' values with median = 1.0
✅ No invalid values found in 'unit_price'
✅ No invalid values found in 'discount_percent'
🔧 Replaced 1 invalid 'total_amount' values with median = 261.55


In [52]:
def numeric_summary(df, label):
    print(f"\n📊 Numeric Summary ({label})")
    display(
        df[DATASET_CONFIGS["main_dataset"]["numeric_columns"]]
        .describe()
        .T
        .style.format(precision=2)
    )

# Show before cleaning
numeric_summary(df_raw, "Before Cleaning")

# Show after cleaning
numeric_summary(df, "After Cleaning")



📊 Numeric Summary (Before Cleaning)


,count,mean,std,min,25%,50%,75%,max
quantity,801440.00,1.57,1.16,-9.00,1.00,1.00,2.00,9.00
unit_price,801440.00,376.63,469.91,3.88,73.48,200.32,428.82,2229.43
discount_percent,801440.00,2.24,5.80,0.00,0.00,0.00,0.00,25.00
total_amount,801440.00,658.88,3498.59,3.88,92.55,261.55,633.56,1255612.50



📊 Numeric Summary (After Cleaning)


,count,mean,std,min,25%,50%,75%,max
quantity,785399.00,1.57,1.16,1.00,1.00,1.00,2.00,9.00
unit_price,785399.00,376.64,469.89,3.88,73.50,200.32,428.82,2229.43
discount_percent,785399.00,2.25,5.80,0.00,0.00,0.00,0.00,25.00
total_amount,785399.00,657.41,3217.06,3.88,92.55,261.55,633.56,549722.32


## 2.3 Standardising Categorical Values

In this step, I will standardise the categorical columns to ensure consistency across the dataset.  
Due to data entry variations or intentional corruptions, the same category might appear in multiple forms (e.g., `"North"`, `"N"`, `"Northern"`).  
I will focus on the following columns:  
- `customer_segment`  
- `product_category`  
- `region`  
- `sales_channel`  

The `region` column will be cleaned using a mapping dictionary derived from the original business scenario. This will reduce the number of unique region values from 25 to the expected 5 standard values:  
**North, South, East, West, Central**.


In [53]:
# --- Step 2.3: Standardising Categorical Values ---

# Expected values dictionary (from business scenario)
EXPECTED_CATEGORIES = {
    "customer_segment": {"Budget", "Premium", "Standard"},
    "product_category": {"Electronics", "Clothing", "Home & Garden", "Sports & Outdoors", "Books & Media"},
    "region": {"North", "South", "East", "West", "Central"},
    "sales_channel": {"Online", "Retail Store", "Phone Orders", "Mobile App"}
}

# Mapping dictionary for region standardisation
REGION_MAPPING = {
    # Central
    "c": "Central", "central": "Central", "centre": "Central",
    # East
    "e": "East", "east": "East", "eastern": "East",
    # North
    "n": "North", "north": "North", "northern": "North",
    # South
    "s": "South", "south": "South", "southern": "South",
    # West
    "w": "West", "west": "West", "western": "West"
}

# Apply standardisation
df["region"] = df["region"].str.strip().str.lower().map(REGION_MAPPING)

# Store unique values before cleaning
unique_before = {col: df[col].nunique() for col in DATASET_CONFIGS["main_dataset"]["categorical_columns"]}

# Apply standardisation
df['region'] = df['region'].map(REGION_MAPPING).fillna(df['region'])

# Store unique values after cleaning
unique_after = {col: df[col].nunique() for col in DATASET_CONFIGS["main_dataset"]["categorical_columns"]}

# Logging changes
for col in DATASET_CONFIGS["main_dataset"]["categorical_columns"]:
    print(f"🔧 Column '{col}': {unique_before[col]} → {unique_after[col]} unique values after standardisation")
    
    # --- Validation for each categorical column ---
    actual_values = set(df[col].unique())
    if actual_values != EXPECTED_CATEGORIES[col]:
        print(f"⚠️ Warning: {col} values do not match expected set.\nFound: {actual_values}")
    else:
        print(f"✅ {col} values match expected set.")


🔧 Column 'customer_segment': 3 → 3 unique values after standardisation
✅ customer_segment values match expected set.
🔧 Column 'product_category': 5 → 5 unique values after standardisation
✅ product_category values match expected set.
🔧 Column 'region': 5 → 5 unique values after standardisation
✅ region values match expected set.
🔧 Column 'sales_channel': 4 → 4 unique values after standardisation
✅ sales_channel values match expected set.


## 2.4 – Statistical Outlier Detection (IQR Method)

In this step, we identify potential statistical outliers in our numeric columns using the Interquartile Range (IQR) method.  
Unlike business rule validation, which is based on predefined acceptable ranges, the IQR method looks for values that lie significantly outside the typical spread of the data.  

- **Why keep outliers?**  
  In many business datasets, extreme values may be valid (e.g., unusually high purchases during sales events).  
  For this project, we will **detect but not modify** these values to preserve the original distribution for further analysis.

**Steps:**
1. For each numeric column:
   - Calculate Q1 (25th percentile) and Q3 (75th percentile).
   - Compute IQR = Q3 - Q1.
   - Define outlier bounds as:  
     - Lower bound = Q1 - 1.5 × IQR  
     - Upper bound = Q3 + 1.5 × IQR  
   - Count how many values fall outside this range.
2. Display summary statistics before and after detection (no changes applied).


In [54]:
def detect_statistical_outliers(df, numeric_columns):
    """
    Detects statistical outliers in numeric columns using the IQR method.
    Outliers are NOT removed or modified — only counted for reference.
    """
    for col in numeric_columns:
        # Calculate Q1, Q3 and IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Detect outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        count_outliers = outliers.shape[0]

        print(f"✅ No invalid values found in '{col}'")
        print(f"📊 Statistical outliers detected in '{col}': {count_outliers} values (not modified)")
        print("-" * 50)

# Run outlier detection
detect_statistical_outliers(
    df,
    DATASET_CONFIGS["main_dataset"]["numeric_columns"]
)

# Show numeric summary before/after (no change expected)
print("\n📊 Numeric Summary (Before & After - same since we didn't modify data)")
display(df[DATASET_CONFIGS["main_dataset"]["numeric_columns"]].describe())


✅ No invalid values found in 'quantity'
📊 Statistical outliers detected in 'quantity': 23579 values (not modified)
--------------------------------------------------
✅ No invalid values found in 'unit_price'
📊 Statistical outliers detected in 'unit_price': 85146 values (not modified)
--------------------------------------------------
✅ No invalid values found in 'discount_percent'
📊 Statistical outliers detected in 'discount_percent': 117485 values (not modified)
--------------------------------------------------
✅ No invalid values found in 'total_amount'
📊 Statistical outliers detected in 'total_amount': 87989 values (not modified)
--------------------------------------------------

📊 Numeric Summary (Before & After - same since we didn't modify data)


,quantity,unit_price,discount_percent,total_amount
count,785399.000000,785399.000000,785399.000000,785399.000000
mean,1.569160,376.636532,2.245034,657.410641
std,1.156608,469.890756,5.799624,3217.058498
min,1.000000,3.880000,0.000000,3.880000
25%,1.000000,73.500000,0.000000,92.550000
50%,1.000000,200.320000,0.000000,261.550000
75%,2.000000,428.820000,0.000000,633.560000
max,9.000000,2229.430000,25.000000,549722.319745


## 2.5 Date Parsing & Validation

In this step, I will **parse and validate all date columns** in the dataset.  
The aim is to ensure that:  

1. All date values are converted into proper `datetime` objects.  
2. Any invalid date formats or parsing errors are detected and removed.  
3. All dates fall within the **business rule date range**:  
   **`2020-01-01` to `2024-12-31`**.  

For each date column, I will:  
- Convert it to `datetime` (invalid entries become `NaT`).  
- Count and display the number of invalid dates.  
- Identify dates outside the defined range.  
- Remove all invalid or out-of-range entries.  

This ensures that **time-based analysis** in later steps will be accurate and free from corrupted date values.


In [55]:
# 2.5 Date Validation & Cleaning

from datetime import datetime

def validate_and_clean_dates(df, date_columns, rules):
    """
    Validates and cleans date columns based on business rules.
    - Removes rows with out-of-range dates
    - Converts to datetime format
    """
    for col in date_columns:
        print(f"\n🗓 Processing date column: '{col}'")
        
        # Convert to datetime safely
        df[col] = pd.to_datetime(df[col], errors='coerce')

        # Drop NaT values
        before_count = len(df)
        df = df.dropna(subset=[col])
        after_count = len(df)
        dropped_nulls = before_count - after_count
        if dropped_nulls > 0:
            print(f"🗑 Removed {dropped_nulls} rows with invalid {col}")

        # Apply range filtering
        min_date, max_date = pd.to_datetime(rules["date_range"])
        out_of_range_mask = (df[col] < min_date) | (df[col] > max_date)
        out_of_range_count = out_of_range_mask.sum()
        
        if out_of_range_count > 0:
            df = df[~out_of_range_mask]
            print(f"⚠️ Found {out_of_range_count} out-of-range dates in '{col}'")
            print(f"🗑 Removed {out_of_range_count} rows from '{col}' after validation")

    print("\n✅ Date columns validated and parsed successfully")
    return df

# Apply function
df = validate_and_clean_dates(
    df,
    DATASET_CONFIGS["main_dataset"]["date_columns"],
    BUSINESS_RULES
)



🗓 Processing date column: 'transaction_date'
⚠️ Found 391 out-of-range dates in 'transaction_date'
🗑 Removed 391 rows from 'transaction_date' after validation

✅ Date columns validated and parsed successfully


## 2.6 Duplicate Detection & Removal

Duplicate records can occur due to **system glitches**, **manual errors**, or **data integration issues**.  
If not removed, they can **inflate totals**, **skew analysis**, and **mislead business decisions**.

In this step, we handle two types of duplicates:

1. **Exact duplicates** – Rows that are identical across all columns.
2. **Soft duplicates** – Rows that are identical across all columns **except unique identifiers** (e.g., `transaction_id`).  
   These often occur when the same transaction is recorded multiple times with a different ID suffix, as in our dataset (`_DUP` entries from the corruption process).

**Approach:**
- First, detect and remove **exact duplicates**.
- Then, detect and remove **soft duplicates** by ignoring ID columns during comparison.
- Report the number of duplicates found and removed.
- Confirm the final row count after cleaning.

In [56]:
# ==========================================
# 2.6 Duplicate Detection & Removal
# ==========================================

def remove_duplicates(df, id_columns):
    initial_count = len(df)
    
    # --- Step 1: Remove exact duplicates ---
    exact_dupes = df.duplicated()
    exact_dupe_count = exact_dupes.sum()
    df = df.drop_duplicates()
    
    print(f"🗑 Removed {exact_dupe_count:,} exact duplicate rows")
    
    # --- Step 2: Remove soft duplicates (ignore ID columns) ---
    non_id_cols = [col for col in df.columns if col not in id_columns]
    soft_dupes = df.duplicated(subset=non_id_cols)
    soft_dupe_count = soft_dupes.sum()
    df = df.drop_duplicates(subset=non_id_cols)
    
    print(f"🗑 Removed {soft_dupe_count:,} soft duplicate rows (ignoring IDs)")
    
    # --- Final dataset size ---
    final_count = len(df)
    removed_total = initial_count - final_count
    print(f"✅ Total duplicates removed: {removed_total:,}")
    print(f"📊 Final dataset size: {final_count:,} rows")
    
    return df

# Apply duplicate removal
df = remove_duplicates(df, DATASET_CONFIGS["main_dataset"]["id_columns"])


🗑 Removed 0 exact duplicate rows
🗑 Removed 7,720 soft duplicate rows (ignoring IDs)
✅ Total duplicates removed: 7,720
📊 Final dataset size: 777,288 rows


## 2.7 – Export Final Clean Dataset

After applying all data cleaning steps — including handling missing values, correcting invalid entries, standardising categorical values, validating dates, and removing duplicates — the dataset is now ready for analysis and integration into other tools.

Exporting the cleaned dataset to a CSV file ensures:

- **Reusability**: Can be imported into BI dashboards, SQL databases, or ML pipelines without re-running the cleaning script.
- **Traceability**: Acts as a permanent record of the cleaned version for reproducibility.
- **Performance**: Avoids repeating time-consuming cleaning steps in future analyses.

We will now save the final DataFrame to disk.


In [59]:
# COMPREHENSIVE VALIDATION
print(f"\n📋 COMPREHENSIVE VALIDATION")
print("=" * 40)

# Check all business rules
validation_results = {}

# Date range validation
date_min = df['transaction_date'].min()
date_max = df['transaction_date'].max()
expected_min = pd.to_datetime(BUSINESS_RULES["date_range"][0])
expected_max = pd.to_datetime(BUSINESS_RULES["date_range"][1])

date_valid = (date_min >= expected_min) and (date_max <= expected_max)
validation_results['date_range_valid'] = date_valid
print(f"   📅 Date range: {date_min.date()} to {date_max.date()} {'✅' if date_valid else '❌'}")

# Quantity validation
qty_min = df['quantity'].min()
qty_max = df['quantity'].max()
qty_valid = (qty_min >= BUSINESS_RULES["quantity_range"][0])
validation_results['quantity_valid'] = qty_valid
print(f"   📦 Quantity range: {qty_min} to {qty_max} {'✅' if qty_valid else '❌'}")

# Amount validation
amt_min = df['total_amount'].min()
amt_max = df['total_amount'].max()
amt_valid = (amt_min >= BUSINESS_RULES["amount_range"][0]) and (amt_max <= BUSINESS_RULES["amount_range"][1])
validation_results['amount_valid'] = amt_valid
print(f"   💰 Amount range: ${amt_min:.2f} to ${amt_max:,.2f} {'✅' if amt_valid else '❌'}")

# Region validation
unique_regions = set(df['region'].unique())
expected_regions = set(BUSINESS_RULES["valid_regions"])
regions_valid = unique_regions.issubset(expected_regions)
validation_results['regions_valid'] = regions_valid
print(f"   🌍 Regions: {len(unique_regions)} unique {'✅' if regions_valid else '❌'}")

# Duplicate validation
duplicates_remaining = df.duplicated(subset=['transaction_id']).sum()
duplicates_valid = (duplicates_remaining == 0)
validation_results['no_duplicates'] = duplicates_valid
print(f"   🔄 Duplicates: {duplicates_remaining} remaining {'✅' if duplicates_valid else '❌'}")

# FINAL QUALITY SCORE
print(f"\n🏆 FINAL QUALITY ASSESSMENT")
print("=" * 40)
missing_customers_final = df['customer_id'].isnull().sum()
total_records_final = len(df)

# Only missing customer IDs remain as "issues" (business decision to keep)
remaining_issues = missing_customers_final
final_quality_score = max(0, 100 - (remaining_issues / total_records_final * 100))

print(f"   Total records: {total_records_final:,}")
print(f"   Remaining issues: {remaining_issues:,} (missing customer IDs)")
print(f"   Final quality score: {final_quality_score:.1f}%")
print(f"   Quality improvement: {final_quality_score - 96.6:.1f} percentage points")

# EXPORT CLEANED DATA
print(f"\n💾 EXPORTING CLEANED DATA")
print("=" * 30)

# Remove the flag column before export (internal use only)
export_df = df.drop(columns=['missing_customer_flag'], errors='ignore')

# Export to processed folder
output_file = DATA_PATHS["processed"] + "sales_cleaned.csv"
export_df.to_csv(output_file, index=False)

file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
print(f"   ✅ Exported: {output_file}")
print(f"   📊 Records: {len(export_df):,}")
print(f"   💾 File size: {file_size_mb:.1f} MB")

# CREATE CLEANING SUMMARY REPORT
cleaning_summary = {
    'cleaning_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'original_records': int(len(sales_df)),  # Convert to regular int
    'final_records': int(len(export_df)),
    'data_retention_pct': round((len(export_df) / len(sales_df)) * 100, 1),
    'quality_improvement': round(final_quality_score - 96.6, 1),
    'final_quality_score': round(final_quality_score, 1),
    'issues_resolved': {
        'duplicates_removed': int(cleaning_log.get('duplicates_removed', 0)),
        'future_dates_corrected': int(cleaning_log.get('future_dates_corrected', 0)),
        'regions_standardized': int(cleaning_log.get('regions_standardized', 0)),
        'outliers_capped': int(cleaning_log.get('outliers_capped', 0)),
        'negative_quantities_fixed': int(negative_count)
    },
    'validation_results': {k: bool(v) for k, v in validation_results.items()}  # Convert to regular bool
}

# Save summary
summary_file = DATA_PATHS["processed"] + "cleaning_summary.json"
with open(summary_file, 'w') as f:
    json.dump(cleaning_summary, f, indent=2)

print(f"   ✅ Cleaning summary saved: {summary_file}")

print(f"\n🎉 DATA CLEANING FRAMEWORK COMPLETE!")
print("=" * 50)
print(f"✅ Professional-grade data quality framework implemented")
print(f"✅ {final_quality_score:.1f}% quality score achieved")
print(f"✅ Framework ready for reuse on any project")
print(f"✅ Clean data ready for ML/AI components")



📋 COMPREHENSIVE VALIDATION
   📅 Date range: 2022-01-01 to 2024-12-31 ✅
   📦 Quantity range: 1 to 9 ✅


KeyError: 'amount_range'